In [1]:
import os
import logging
import networkx as nx
import numpy as np
#import seaborn as sns
import random
import math
from scipy.stats import truncnorm
import time
import copy
import matplotlib.pyplot as plt
from random import sample
import itertools

data_name = "auth"
NCCPATH =  '../data/LANL/TCG_LANL.txt'
SEED = 1
ASG = "./anomaly_associate_graph.txt"


sample_user = 2000  # 2000 normal users

In [2]:
red_team = './nredteam.txt'
anomaly_user = set()
with open(red_team, 'r', encoding='utf-8') as fi:
    while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        anomaly_user.add(l[1])

In [8]:

def sample_users(file_path, sample_user):
    with open(file_path, "r") as f:
        line = f.readline()
        sampled_users = set()
        num_lines_read = 0
        start = time.time()
        while True:
            line = f.readline()
            if not line:
                break
            l = line.strip().split(',')
            if len(sampled_users) < sample_user and l[1] not in anomaly_user:
                sampled_users.add(l[1])
            elif(l[1] not in anomaly_user and l[1] not in sampled_users):
                index_to_replace = random.randint(0, sample_user - 1)
                list(sampled_users)[index_to_replace] = l[1]
            num_lines_read += 1
            if (num_lines_read % 5000000 == 0):
                print(time.time() - start)
    return sampled_users


sampled_users = sample_users(NCCPATH, sample_user)


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.000993490219116211
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0009970664978027344
0.0
0.0
0.0
0.0


In [17]:
def write_lines(file_path,ASG, sample_user):
    with open(file_path, "r") as f:
        line = f.readline()
        num_lines_read = 0
        start = time.time()
        sf = open(ASG, 'a')
        while True:
            line = f.readline()
            if not line:
                break
            l = line.strip().split(',')
            if l[1] in sampled_users or l[1] in anomaly_user:
                sf.write(line)
            num_lines_read += 1
            if (num_lines_read % 3000000 == 0):
                print(time.time() - start)
    return num_lines_read

with open(ASG, 'a') as sf:
    sf.truncate(0)
    sf.close()
num_lines_read = write_lines(NCCPATH,ASG, sample_user)


4.178261995315552
8.160568237304688
12.843894481658936
17.572509050369263
21.878676652908325
26.15847134590149
30.72716212272644
35.106136322021484
39.628639459609985
44.53762459754944
49.06537175178528
53.82852792739868


In [18]:
red_team = '../data/LANL/nredteam.txt'
red_file = set(open(red_team, 'r').readlines())
ASG_events = 0
ASG_node = set()
ASG_anomaly_node = set()
ASG_anomaly_events = 0 
with open(ASG, 'r', encoding='utf-8') as fi:
    while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        ASG_events += 1
        ASG_node.add(l[3])
        ASG_node.add(l[6])
        str_line = str(l[0]) + ',' + str(l[1]) + ',' + str(l[2]) + ',' + str(l[3]) + ',' + str(l[6]) + '\n'
        if str_line in red_file:
            ASG_anomaly_events+=1
            ASG_anomaly_node.add(l[3])
            ASG_anomaly_node.add(l[6])
    fi.close()


In [19]:
print("num of all events:",ASG_events)
print("num of all nodes:",len(ASG_node))
print("num of all anomaly events:",ASG_anomaly_events)
print("num of all anomaly nodes:",len(ASG_anomaly_node))

num of all events: 11839847
num of all nodes: 8548
num of all anomaly events: 702
num of all anomaly nodes: 299
